In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd

# To use selenium webdriver you have to install selenium (pip install selenium) 
# and download the specific driver depending on the browser you are using:
# https://sites.google.com/a/chromium.org/chromedriver/downloads (for chrome).
# Then make sure the downloaded file is in your PATH (e.p. /usr/local/bin for mac).


In [ ]:
# Create the instance of Chrome webdriver.
# browser.get navigates to the page given by the URL.

browser = webdriver.Chrome()
browser.get("https://www.dublinbus.ie/RTPI/Sources-of-Real-Time-Information/?searchtype=route&searchquery=15")

# Get the source code using selenium and beautifulsoup:

html = browser.page_source
soup = bs(html, "lxml")
soup.prettify()
soup


In [ ]:
# Dublin Bus uses a javascript to load the list of bus stops on a route without changing the URL.
# So we have to simulate the click event and capture the source code after.
# Find the link in the source code that lets you view all the stops on the chosen route.
# Use selenium to simulate the click of the 'View Direction' button.

browser.find_element_by_link_text('View Direction').click()


In [ ]:
# Get the source code after the click event.

html_new = browser.page_source
soup = bs(html_new, "lxml")
print(soup.prettify())


In [ ]:
# Find the table that displays all the stops along the route.

data = soup.findAll('table',attrs={'id':'rtpi-results'})

# Find all the rows in the table and add the content to the list 'stops'.

stops = []

for tr in data:
    rows = tr.findAll('td')
    for td in rows:
        stops.append(td.contents[0])


In [ ]:
# Cleans the list for further usage.

def strip_stops(stops):
    '''Function that strips the list objects of whitespaces, new line breaks, and empty list objects'''
    
    stripped = [x.strip('\n') for x in stops]
    stripped = [x.rstrip() for x in stripped]
    stripped = [x.strip(' ') for x in stripped]
    stripped = [x for x in stripped if x != '']
    
    return stripped

stripped = strip_stops(stops)


In [ ]:
# def create_stop_list(list):
    
#     StopID = []
    
#     for i in list[::3]:
#         StopID.append(i)
        
#     StopName = []

#     for i in list[1::3]:
#         StopName.append(i)
        
#     StopNameSpecific = []

#     for i in list[2::3]:
#         StopNameSpecific.append(i)
        
#     return StopID, StopName, StopNameSpecific

#     full_list = []
    
#     full_list.

# full_list = create_stop_list(stripped)
    

In [ ]:
# df1 = pd.DataFrame(full_list)
# # df1 = df1.transpose()
# # df1.columns = ['StopID', 'StopName', 'StopNameSpecific']
# df1

In [ ]:
# Extract the stop numbers and store them in a separate list.

StopID = []

for i in stripped[::3]:

    StopID.append(i)


In [ ]:
# Extract the stop names and store them in a separate list.

StopName = []

for i in stripped[1::3]:

    StopName.append(i)


In [ ]:
# Extract the more specific names and store them in a separate list.

StopNameSpecific = []

for i in stripped[2::3]:
    StopNameSpecific.append(i)


In [ ]:
# Reads the individual lists into a joint dataframe.

df = pd.DataFrame(
    {'StopID': StopID,
     'StopName': StopName,
     'StopNameSpecific': StopNameSpecific
    })

df.head()


In [ ]:
df.to_csv("bus_data/cleaned_data/line15_stop_list_today.csv")

In [ ]:
browser.quit()

## Storing stops in DB: